<a href="https://colab.research.google.com/github/softmurata/AudioML/blob/main/voiceconv/so_vits_svc_v5_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Docs

[github](https://github.com/PlayVoice/so-vits-svc-5.0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installation

In [ ]:
!git clone https://github.com/PlayVoice/so-vits-svc-5.0.git

In [ ]:
%cd /content/so-vits-svc-5.0
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple -r requirements.txt

Download Pretrained weights

In [ ]:
%cd /content
!gdown --fuzzy https://drive.google.com/file/d/1UPjQ2LVSIt3o-9QMKMJcdzT8aZRZCI-E/view?usp=drive_link

In [ ]:
!wget https://openaipublic.azureedge.net/main/whisper/models/81f7c96c852ee8fc832187b0132e569d6c3065a3252ed18e56effd0b6a73e524/large-v2.pt

In [ ]:
!wget https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt

In [ ]:
!wget https://github.com/maxrmorrison/torchcrepe/raw/master/torchcrepe/assets/full.pth

In [ ]:
!wget https://github.com/PlayVoice/so-vits-svc-5.0/releases/download/5.0/sovits5.0.pretrain.pth

Put them

In [ ]:
!cp /content/best_model.pth.tar /content/so-vits-svc-5.0/speaker_pretrain/
!cp /content/large-v2.pt /content/so-vits-svc-5.0/whisper_pretrain/
!cp /content/hubert-soft-0d54a1f4.pt /content/so-vits-svc-5.0/hubert_pretrain/
!cp /content/full.pth /content/so-vits-svc-5.0/crepe/assets/
!cp /content/sovits5.0.pretrain.pth  /content/so-vits-svc-5.0/vits_pretrain/

Check inference

In [ ]:
%cd /content/so-vits-svc-5.0
!python svc_inference.py --config configs/base.yaml --model ./vits_pretrain/sovits5.0.pretrain.pth --spk ./configs/singers/singer0001.npy --wave test.wav

In [ ]:
from IPython.display import Audio, display

sound_boo = "/content/so-vits-svc-5.0/svc_out.wav"
display(Audio(sound_boo, autoplay=True))

Dataset Preparation

In [ ]:
!mkdir /content/dataset_raw

In [ ]:
!mkdir /content/dataset_raw/speaker0
!cp -r /content/drive/MyDrive/anya/* /content/dataset_raw/speaker0/

In [ ]:
!cp -r /content/dataset_raw /content/so-vits-svc-5.0/

In [ ]:
%cd /content/so-vits-svc-5.0
!python svc_preprocessing.py -t 1

Training

In [ ]:
# 700回くらい学習させるといい感じ！保存する重みのintervalを減らさないとcolabのストレージが足りなくなるので注意!
%cd /content/so-vits-svc-5.0
!python svc_trainer.py -c configs/base.yaml -n sovits5.0

Export Model

In [ ]:
%cd /content/so-vits-svc-5.0
!python svc_export.py --config configs/base.yaml --checkpoint_path /content/so-vits-svc-5.0/chkpt/sovits5.0/sovits5.0_0700.pt

Not Adjust Inference

In [ ]:
# めっちゃ時間かかる、、
# あらかじめ作成しておいた上でembeddingで似たような文章を取得して返すみたいな感じにするしかない、、
# %cd /content/so-vits-svc-5.0
!python svc_inference.py --config configs/base.yaml --model sovits5.0.pth --spk ./data_svc/singer/speaker0.spk.npy --wave /content/so-vits-svc-5.0/anmarimuzukashiku_01.wav --shift 0

Drive inference

In [ ]:
%cd /content/drive/MyDrive/so-vits-svc-5.0
!python svc_inference.py --config configs/base.yaml --model sovits5.0.pth --spk ./data_svc/singer/speaker0.spk.npy --wave /content/answerfiles/002.wav --shift 0

In [ ]:
from IPython.display import Audio, display

sound_boo = "/content/case1/002.wav"
display(Audio(sound_boo, autoplay=True))

In [ ]:
!cp -r /content/so-vits-svc-5.0/ /content/drive/MyDrive/

In [ ]:
!rm -rf /content/drive/MyDrive/so-vits-svc-5.0/chkpt

Base japanese wav creation

In [ ]:
!pip install -q pyopenjtalk-prebuilt
!pip install -q pyopenjtalk[marine]

In [ ]:
import pyopenjtalk
from scipy.io import wavfile
import numpy as np

x, sr = pyopenjtalk.tts("こういうことはしっかりと上司に連絡したほうがいいます。しっかりするます。明日は頑張るます。", run_marine=True)

wavfile.write("/content/test.wav", sr, x.astype(np.int16))

from IPython.display import Audio, display
display(Audio("/content/test.wav", autoplay=True))

音声を作成するJob

In [ ]:
%cd /content/drive/MyDrive/so-vits-svc-5.0

import pyopenjtalk
from scipy.io import wavfile
import numpy as np
import os

# ユースケース1: 新規顧客獲得
case1 = [
    {"Q": "営業: こんにちは、村田さん。弊社のサービスに興味を持っていただきありがとうございます。まず最初に、お手伝いできることや特にお困りごとはありますか？",
     "A": "お客様: はい、最近村田さんに課題を抱えていて、それに対する解決策を模索しています。"},
    {"Q": "営業: それは理解できます。お聞かせいただければと思いますが、現在の課題の具体的なポイントは何でしょうか？",
     "A": "お客様: 村田さんが効率的に機能していないことが主な課題です。"},
    {"Q": "営業: なるほど、それは重要なポイントですね。弊社のサービスは、村田さんの効率向上に特に焦点を当てており、その点でお手伝いできるかもしれません。どのような要件や期待をお持ちでしょうか？",
     "A": "お客様: 具体的な要件としては、村田さんがスムーズに行えることと、村田さんの改善案があれば教えていただきたいです。"}
]

# ユースケース2: 既存顧客へのアップセル
case2 = [
    {"Q": "営業: お世話になっております、村田さん。これまでのご利用ありがとうございます。今回は、新たな機能やアップグレードについてお知らせがあります。お時間いただけますでしょうか？",
     "A": "お客様: はい、どのような内容ですか？"},
    {"Q": "営業: 弊社では最近、〇〇機能を追加しました。これにより、〇〇の利便性が向上し、業務の効率化が期待できます。ご興味がありましたら、詳細をご案内させていただけますか？",
     "A": "お客様: 興味がありますね。具体的にどのような変更があるのでしょうか？"},
    {"Q": "営業: それでは、〇〇機能の詳細や導入メリットについてお話しいたしますね。",
     "A": "お客様: 聞きたいことがあれば質問させてください。"}
]

# ユースケース3: フィードバック収集
case3 = [
    {"Q": "営業: こんにちは、〇〇さん。ご愛顧いただきありがとうございます。今日は、弊社のサービスに関するフィードバックをお伺いしたく存じます。ご意見やご感想がありましたらお聞かせいただけますか？",
     "A": "お客様: あまり大きな問題はないですが、時々〇〇に関する使い勝手が気になります。"},
    {"Q": "営業: それは貴重なご意見ですね。具体的にどのような点が気になりましたか？",
     "A": "お客様: 〇〇の操作が少し複雑で、もっと直感的になればいいなと思います。"},
    {"Q": "営業: 了解しました。おっしゃる通り、使いやすさが重要ですね。今後の開発で改善できるように努めてまいります。他にも何かありましたらお気軽におっしゃってください。",
     "A": "お客様: ありがとうございます。他には特に問題ないですが、改善点があれば随時お知らせいたします。"}
]

# 各ユースケースを辞書でまとめる
qa_data = {"case1": case1, "case2": case2, "case3": case3}

import os

for name, c in qa_data.items():
  os.makedirs(f"/content/{name}", exist_ok=True)

  for idx, item in enumerate(c):
    answer = item["A"]

    x, sr = pyopenjtalk.tts(answer, run_marine=True)

    wavfile.write("/content/test.wav", sr, x.astype(np.int16))

    !python svc_inference.py --config configs/base.yaml --model sovits5.0.pth --spk ./data_svc/singer/speaker0.spk.npy --wave /content/test.wav --shift 0

    os.system(f"cp /content/drive/MyDrive/so-vits-svc-5.0/svc_out.wav /content/{name}/{str(idx).zfill(3)}.wav")

In [ ]:
!rm -rf /content/*.zip
!zip -r /content/case1.zip /content/case1/
!zip -r /content/case2.zip /content/case2/
!zip -r /content/case3.zip /content/case3/

最近棒探索を利用したchatbot応答システム。

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('pkshatech/GLuCoSE-base-ja')

In [ ]:
conversations = [
    {"question": "最近、数学が難しく感じるんだ。", "answer": "分からないところを教えてくれれば、一緒に解決してみよう。"},
    {"question": "方程式の解き方がよくわからないんだけど。", "answer": "まずは具体的な方程式を教えてくれると助かるよ。"},
    {"question": "三角関数が難しいなあ。", "answer": "特にどの部分が難しいと感じてるの？"},
    {"question": "微分と積分の違いが分からないんだけど。", "answer": "微分は変化の率を求める操作で、積分は面積や合計を求める操作なんだよ。"},
    {"question": "確率ってどうやって計算するの？", "answer": "確率の計算は試行回数や好ましい結果の場合の数を考えることから始めるんだ。"},
    {"question": "複素数って何？", "answer": "複素数は実部と虚部からなる数で、iは虚数単位だよ。"},
    {"question": "平方完成って難しいんだけど。", "answer": "平方完成は二次方程式を解くための方法で、具体的な例を一緒に見てみよう。"},
    {"question": "三平方の定理を使った問題が解けないんだ。", "answer": "三平方の定理は直角三角形において、各辺の長さに関する定理だよ。どんな問題があるの？"},
    {"question": "集合の演算ってどうやるの？", "answer": "和集合や積集合など、集合の演算は要素同士の関係性を考えるものだよ。"},
    {"question": "関数のグラフの描き方が難しい。", "answer": "関数のグラフは、変数の値と関数の値の対応を視覚的に表現するものだよ。"},
    {"question": "行列の掛け算って手順がいまいち分からないな。", "answer": "行列の掛け算は各要素同士の積を計算し、結果の行列を得る操作だよ。"},
    {"question": "三角不等式って何？", "answer": "三角形の辺の長さを軸として、不等式を立てるやつだよ"},
    {"question": "不等式の解き方が分からないんだ。", "answer": "不等式は、変数に対する条件を表すもので、解くと変数の取りうる範囲がわかるよ。"},
    {"question": "数列と級数の違いは？", "answer": "数列は数の並びで、級数は数列の和を指すよ。"},
    {"question": "因数分解ってどうやるの？", "answer": "因数分解は式を因数の積に分解する方法で、共通因数を見つけるとやりやすくなるよ。"},
    {"question": "円の面積の求め方を教えて。", "answer": "円の面積はπr²だよ。半径が分からないと計算できないから、それが分かったら教えてね。"},
    {"question": "直線と曲線の方程式の違いって何？", "answer": "直線の方程式は一次方程式で表され、曲線は二次方程式やそれ以上の次数の方程式になるよ。"},
    {"question": "極座標って何だろう？", "answer": "極座標は、座標を半径と角度で表す方法で、円や複素数の表現に使われるよ。"},
    {"question": "統計って難しいよね。", "answer": "統計はデータを分析する方法で、平均や標準偏差などを計算してデータの特徴を見つけるんだ。"},
    {"question": "微分方程式って何？", "answer": "流体力学とかでよく出てくる方程式で、微分を基にして時間方向の方程式を立てる感じだよ"}
]

questions = []
for item in conversations:
  questions.append(item["question"])
question_embeddings = model.encode(questions)

In [ ]:
import numpy as np

def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return 0
    else:
        return dot_product / (magnitude_v1 * magnitude_v2)

In [ ]:
query = "級数について教えて"
source_embeddings = model.encode([query])

cos_sim_value_list = []
for t in question_embeddings:
  cos_sim_value = cosine_similarity(t, source_embeddings.reshape(-1))
  cos_sim_value_list.append(cos_sim_value)

top_id = np.argsort(cos_sim_value_list)[::-1][0]
print(conversations[top_id]["answer"])

数列は数の並びで、級数は数列の和を指すよ。


WebUI

In [ ]:
!pip install -q gradio

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from scipy.io import wavfile

def wav_to_np_array(file_path):
    # WAV ファイルの読み込み
    sample_rate, data = wavfile.read(file_path)

    # 複数のチャンネルがある場合、1つに統合
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)

    return sample_rate, data


def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return 0
    else:
        return dot_product / (magnitude_v1 * magnitude_v2)

model = SentenceTransformer('pkshatech/GLuCoSE-base-ja')

conversations = [
    {"question": "最近、数学が難しく感じるんだ。", "answer": "分からないところを教えてくれれば、一緒に解決してみよう。"},
    {"question": "方程式の解き方がよくわからないんだけど。", "answer": "まずは具体的な方程式を教えてくれると助かるよ。"},
    {"question": "三角関数が難しいなあ。", "answer": "特にどの部分が難しいと感じてるの？"},
    {"question": "微分と積分の違いが分からないんだけど。", "answer": "微分は変化の率を求める操作で、積分は面積や合計を求める操作なんだよ。"},
    {"question": "確率ってどうやって計算するの？", "answer": "確率の計算は試行回数や好ましい結果の場合の数を考えることから始めるんだ。"},
    {"question": "複素数って何？", "answer": "複素数は実部と虚部からなる数で、iは虚数単位だよ。"},
    {"question": "平方完成って難しいんだけど。", "answer": "平方完成は二次方程式を解くための方法で、具体的な例を一緒に見てみよう。"},
    {"question": "三平方の定理を使った問題が解けないんだ。", "answer": "三平方の定理は直角三角形において、各辺の長さに関する定理だよ。どんな問題があるの？"},
    {"question": "集合の演算ってどうやるの？", "answer": "和集合や積集合など、集合の演算は要素同士の関係性を考えるものだよ。"},
    {"question": "関数のグラフの描き方が難しい。", "answer": "関数のグラフは、変数の値と関数の値の対応を視覚的に表現するものだよ。"},
    {"question": "行列の掛け算って手順がいまいち分からないな。", "answer": "行列の掛け算は各要素同士の積を計算し、結果の行列を得る操作だよ。"},
    {"question": "三角不等式って何？", "answer": "三角形の辺の長さを軸として、不等式を立てるやつだよ"},
    {"question": "不等式の解き方が分からないんだ。", "answer": "不等式は、変数に対する条件を表すもので、解くと変数の取りうる範囲がわかるよ。"},
    {"question": "数列と級数の違いは？", "answer": "数列は数の並びで、級数は数列の和を指すよ。"},
    {"question": "因数分解ってどうやるの？", "answer": "因数分解は式を因数の積に分解する方法で、共通因数を見つけるとやりやすくなるよ。"},
    {"question": "円の面積の求め方を教えて。", "answer": "円の面積はπr²だよ。半径が分からないと計算できないから、それが分かったら教えてね。"},
    {"question": "直線と曲線の方程式の違いって何？", "answer": "直線の方程式は一次方程式で表され、曲線は二次方程式やそれ以上の次数の方程式になるよ。"},
    {"question": "極座標って何だろう？", "answer": "極座標は、座標を半径と角度で表す方法で、円や複素数の表現に使われるよ。"},
    {"question": "統計って難しいよね。", "answer": "統計はデータを分析する方法で、平均や標準偏差などを計算してデータの特徴を見つけるんだ。"},
    {"question": "微分方程式って何？", "answer": "流体力学とかでよく出てくる方程式で、微分を基にして時間方向の方程式を立てる感じだよ"}
]

questions = []
for item in conversations:
  questions.append(item["question"])
question_embeddings = model.encode(questions)


In [ ]:
import gradio as gr

blocks = gr.Blocks()

message_history = []

with blocks as demo:
  with gr.Row():
    gr.Markdown("## Mathmatic Chatbot")

  with gr.Column():
    chatbot = gr.Chatbot()

    with gr.Row():
      input = gr.Textbox(label="Input")

    audio = gr.Audio(label="Output Audio", elem_id="tts-audio", autoplay=True)

    def chat_func(query):
      global message_history

      message_history.append({
          "role": "user",
          "content": query
      })
      source_embeddings = model.encode([query])

      cos_sim_value_list = []
      for t in question_embeddings:
        cos_sim_value = cosine_similarity(t, source_embeddings.reshape(-1))
        cos_sim_value_list.append(cos_sim_value)

      top_id = np.argsort(cos_sim_value_list)[::-1][0]
      answer = conversations[top_id]["answer"]

      message_history.append({
          "role": "assistant",
          "content": answer
      })

      # WAV ファイルのパス
      file_path = f'/content/answerfiles/{str(top_id).zfill(3)}.wav'
      # WAV ファイルを numpy 配列に変換
      sample_rate, audio_data = wav_to_np_array(file_path)

      return [(message_history[i]["content"], message_history[i + 1]["content"]) for i in range(0, len(message_history) - 1, 2)], (sample_rate, audio_data)


    input.submit(
        chat_func,
        inputs=[input],
        outputs=[chatbot, audio]
    )

    input.submit(
        fn= lambda: "",
        inputs=None,
        outputs=input
    )

demo.launch(debug=True)